In [0]:
!pip install -U tiktoken openpyxl openai

  Obtaining dependency information for tiktoken from https://files.pythonhosted.org/packages/01/c4/c4a4360de845217b6aa9709c15773484b50479f36bb50419c443204e5de9/tiktoken-0.8.0-cp311-cp311-manylinux_2_17_x86_64.manylinux2014_x86_64.whl.metadata
  Obtaining dependency information for openpyxl from https://files.pythonhosted.org/packages/c0/da/977ded879c29cbd04de313843e76868e6e13408a94ed6b987245dc7c8506/openpyxl-3.1.5-py2.py3-none-any.whl.metadata
  Using cached openpyxl-3.1.5-py2.py3-none-any.whl.metadata (2.5 kB)
  Obtaining dependency information for openai from https://files.pythonhosted.org/packages/9a/b6/2e2a011b2dc27a6711376808b4cd8c922c476ea0f1420b39892117fa8563/openai-1.61.1-py3-none-any.whl.metadata
  Obtaining dependency information for et-xmlfile from https://files.pythonhosted.org/packages/c1/8b/5fe2cc11fee489817272089c4203e679c63b570a5aaeb18d852ae3cbba6a/et_xmlfile-2.0.0-py3-none-any.whl.metadata
  Using cached et_xmlfile-2.0.0-py3-none-any.whl.metadata (2.7 kB)
  Obtaining d

In [0]:
dbutils.library.restartPython()

In [0]:
import pandas as pd
import os
import json
import yaml
import requests
import openai
import tiktoken
import re

In [0]:
class RegexConstants:
    """
    Regex constants
    """
    INVALID_VALUES_REGEX = "(<[^>]*>)|(options:.*)|(empty)|(none)|(null)|(value_needed)|(undefined)|(string)|(n/a)|(human)|(you)|(default)|(default_value_if_not_filled)"
    IDENTIFY_NEW_LINE = r"#.*\n"

def extract_json(resp: str, inner_json: bool = True):
    """
    Function to extract the stringified json structure which is part of the bot's response.
    Bot response is designed to have all the fields and their filled or unfilled values(options or default values)
    in each turn.
    :param resp: bot response
    :param inner_json: extract only inner json or full json
    :return: json with accepted values (which are other than the default values/options)
    """
    if inner_json:
        json_part = resp[resp.rfind('{'):]
        json_part = json_part[:json_part.find('}') + 1]
    else:
        json_part = resp[resp.find('{'):]
        json_part = json_part[:json_part.rfind('}') + 1]
    if len(json_part) == 0:
        return None
      
   
    json_part_filtered = re.sub(RegexConstants.IDENTIFY_NEW_LINE, '\n', json_part)
    # print(yaml.load(json_part_filtered, yaml.SafeLoader))
    try:
        extracted_dict = yaml.load(json_part_filtered, yaml.SafeLoader)
        
        return {str(k).strip(): str(v).strip() for k, v in extracted_dict.items() if
                not re.fullmatch(RegexConstants.INVALID_VALUES_REGEX, str(v).lower())}
    except Exception as e:
       
        valid_chunks = dict()
        for line in resp.split("\n"):
            try:
                valid_chunks.update(yaml.load("{" + line + "}", yaml.SafeLoader))
            except Exception:
                continue
        valid_chunks = {str(k).strip(): str(v).strip() for k, v in valid_chunks.items() if
                        not re.fullmatch(RegexConstants.INVALID_VALUES_REGEX, str(v).lower())}
        if valid_chunks != {}:
            
            return valid_chunks
       
        return None

In [0]:
import time
import requests
def get_res_cloudverse_endpoint(messages, temperature, verify=False, model_name="Azure-GPT-4o"):
    os.environ['REQUESTS_CA_BUNDLE'] = os.environ["CUSTOM_CA_PATH"].strip()
    st = time.time()
    res = None
 
    URL = "https://cloudverse.freshworkscorp.com/api/chat"
    req_data = {
        "model": model_name,
        "messages": messages,
        "temperature": temperature
    }
    response = requests.post(URL,
                headers = {
                    "Content-Type": "application/json",
                    "Authorization": f'Bearer {os.environ["CLOUD_VERSE_TOKEN"].strip()}'
                    },
                json=req_data,
                verify=verify)
    
    os.environ['REQUESTS_CA_BUNDLE'] = ""
    #print("Time taken:", time.time() - st, "seconds")
        # res = extract_json(response, inner_json=False)
    # except Exception as e:
        # print("Something went wrong!", e)
    return response.json(), ""

In [0]:
encoding = tiktoken.get_encoding("o200k_base")

In [0]:
def get_token_count(text):
  encoding = tiktoken.get_encoding("o200k_base")
  return len(encoding.encode(text))
def get_encoding(text):
  encoding = tiktoken.get_encoding("o200k_base")
  return encoding.encode(text)

In [0]:
df = pd.read_excel("CleanedFreshTicketsfromCustomers.xlsx")

In [0]:
df = df.dropna(subset=["Subject", "Description"])

In [0]:
prompt_conv_format = """Given below is an unformatted conversation. Format it to make it more readable and LLM friendly:
### Input
{conversation}

### Response Format
- Outout only the formatted text without any additional text or message.
"""

In [0]:
def process_conv(x):
  time.sleep(2)
  messages = [{
    "role": "system",
    "content": prompt_conv_format.format(conversation=x)
  }]
  raw_res = get_res_cloudverse_endpoint(messages=messages, temperature=0.)[0]
  return raw_res

In [0]:
df["text_2_cluster"] = "SUBJECT: " + df["Subject"] + "\n" + "DESCRIPTION: " + df["Description"]

In [0]:
temp_df_2 = df.iloc[100:200].copy()
temp_df_2["formatted_text"] = temp_df_2["text_2_cluster"].apply(lambda x: process_conv(x))

/databricks/python/lib/python3.11/site-packages/urllib3/connectionpool.py:1056: InsecureRequestWarning: Unverified HTTPS request is being made to host 'cloudverse.freshworkscorp.com'. Adding certificate verification is strongly advised. See: https://urllib3.readthedocs.io/en/1.26.x/advanced-usage.html#ssl-warnings
  warnings.warn(
/databricks/python/lib/python3.11/site-packages/urllib3/connectionpool.py:1056: InsecureRequestWarning: Unverified HTTPS request is being made to host 'cloudverse.freshworkscorp.com'. Adding certificate verification is strongly advised. See: https://urllib3.readthedocs.io/en/1.26.x/advanced-usage.html#ssl-warnings
  warnings.warn(
/databricks/python/lib/python3.11/site-packages/urllib3/connectionpool.py:1056: InsecureRequestWarning: Unverified HTTPS request is being made to host 'cloudverse.freshworkscorp.com'. Adding certificate verification is strongly advised. See: https://urllib3.readthedocs.io/en/1.26.x/advanced-usage.html#ssl-warnings
  warnings.warn(
/

In [0]:
temp_df_2.display()

Ticket ID Subject Description Status Created Time Priority Source BU Category Sub-Category Item Full name Email CXG Segment Region Total MRR Freshservice MRR Freshdesk MRR Freshsales MRR Company Name Company Domains text_2_cluster formatted_text 17333450 Conversation with Mark Bagley Mark Bagley how do i list only tickets that have notes 05:55 AM, 14th Jan Freddy To list only tickets that have notes, you can create a custom ticket list view using filters. Apply a filter to include tickets that have notes and save it as a personal or shared view. Here's how: 1. Go to the Tickets tab. 2. Click on the Filters panel. 3. Apply the desired filters to customize your view (in this case, look for tickets with notes). 4. Click the Save View icon. 5. Choose 'Save View As...' to create a new custom view. 6. Select the visibility (Just me for personal view or All agents for shared view). By following these steps, you will have a view that lists only the tickets that have notes. Read More: Understand the Ticket List View 05:55 AM, 14th Jan Freddy Was I helpful? Your feedback matters! 05:55 AM, 14th Jan Mark Bagley No 05:56 AM, 14th Jan Freddy I'm sorry to hear that it wasn't helpful. Let me quickly connect you with a specialist. 05:56 AM, 14th Jan Vinu Hi Mark, hope you are well 05:57 AM, 14th Jan Vinu How may we help you today? 05:57 AM, 14th Jan Mark Bagley Hi. I'm trying to get a list of tickets that have notes. I can't see 'notes' as an option in the filter panel. 05:59 AM, 14th Jan Vinu Thanks for the elaboration. Unfortunately, we do not have the ability to add a filter for notes in Freshdesk. If you wish, I can convert your chat to a ticket and pass this on as a feature request however. 06:02 AM, 14th Jan Mark Bagley That would be very helpful. When you have a lot of tickets that are similar, it is difficult to locate the ones that you have made notes on. 06:04 AM, 14th Jan Vinu Understood, we will pass this on as a feature request. Have a good day 06:09 AM, 14th Jan Take to Team Inbox Waiting on Feature Request 2025-01-14T06:09:40Z Low Chat Freshdesk Analytics Widget Filters What filter to use to analyse required metrics Mark Bagley markb@peacewise.org.au Digital Tier ANZ 238 null 228.0 0.0 peacewise.org.au peacewise.org.au SUBJECT: Conversation with Mark Bagley
DESCRIPTION: Mark Bagley how do i list only tickets that have notes 05:55 AM, 14th Jan Freddy To list only tickets that have notes, you can create a custom ticket list view using filters. Apply a filter to include tickets that have notes and save it as a personal or shared view. Here's how: 1. Go to the Tickets tab. 2. Click on the Filters panel. 3. Apply the desired filters to customize your view (in this case, look for tickets with notes). 4. Click the Save View icon. 5. Choose 'Save View As...' to create a new custom view. 6. Select the visibility (Just me for personal view or All agents for shared view). By following these steps, you will have a view that lists only the tickets that have notes. Read More: Understand the Ticket List View 05:55 AM, 14th Jan Freddy Was I helpful? Your feedback matters! 05:55 AM, 14th Jan Mark Bagley No 05:56 AM, 14th Jan Freddy I'm sorry to hear that it wasn't helpful. Let me quickly connect you with a specialist. 05:56 AM, 14th Jan Vinu Hi Mark, hope you are well 05:57 AM, 14th Jan Vinu How may we help you today? 05:57 AM, 14th Jan Mark Bagley Hi. I'm trying to get a list of tickets that have notes. I can't see 'notes' as an option in the filter panel. 05:59 AM, 14th Jan Vinu Thanks for the elaboration. Unfortunately, we do not have the ability to add a filter for notes in Freshdesk. If you wish, I can convert your chat to a ticket and pass this on as a feature request however. 06:02 AM, 14th Jan Mark Bagley That would be very helpful. When you have a lot of tickets that are similar, it is difficult to locate the ones that you have made notes on. 06:04 AM, 14th Jan Vinu Understood, we will pass this on as a feature request. Have a good day

### Steps
- first filter on product
- then run clustering
- come up with feature+description

In [0]:
first_df = pd.read_excel("Customer_Request_Cluster.xlsx")

In [0]:
first_df.drop(["Cluster"], axis=1, inplace=True)

In [0]:
final_df = pd.concat([first_df, temp_df_2])

In [0]:
final_df.shape

(200, 23)

In [0]:
prompt = """You are am expert product manager working in CRM and ITSM. Given below are some feature requests from customers. Your task is to cluster these together and provide a short description of each cluster. Also output the feature request number that you think belongs to each cluster.

### Input
- Inout contains a numbered list of feature requests.
- Each feature request has a SUBJECT and a DESCRIPTION
- The DESCRIPTION can be one of the below:
  - A paragraph/email describing the feature that the customer wants
  - A chat between the customer and 'a bot or a human agent' where the customer is describing what their issue is and what feature they would want.

### Important Points to Note
- ENSURE each feature request is assigned a cluster.
- Cluster the given data points into meaningful groups based on their semantic similarities. Ensure that the clusters are neither too broad nor overly granular, capturing key distinctions while maintaining coherence within each group. 
- CRITICAL!!! The segmentation should be interpretable and useful for analysis, grouping similar concepts together while avoiding excessive merging of distinct topics. Aim for a level of abstraction that preserves important differences without unnecessary fragmentation.

### Response Format
Always output the below JSOn without any additional text:
{{
  "clusters": [
    {{
      "description": "short description of the cluster",
      "feature_requests": [list of feature requests belonging to this cluster]
    }}
    ]
}}

### Feature Requests
"""

In [0]:
prompt_v2 = """You are an expert product manager specializing in CRM and ITSM. You have been provided with a numbered list of customer feature requests. Each request includes a **SUBJECT** and a **DESCRIPTION**. The **DESCRIPTION may be either a paragraph/email or a chat transcript between the customer and an agent (human or bot). Your objective is to analyze and cluster these feature requests into coherent and meaningful groups that will help prioritize future product features.

### Instructions
1. **Cluster Assignment:**  
   - Ensure every feature request is assigned to exactly one cluster.
   - Group requests based on their semantic similarity. Consider both the **SUBJECT** and the **DESCRIPTION**.
   - Create clusters that are neither too broad (losing important distinctions) nor too narrow (fragmenting similar requests).

2. **Cluster Description:**  
   - For each cluster, provide a concise description summarizing the shared theme or need.
   - Focus on actionable themes that can inform feature development decisions.

3. **Granularity & Interpretability:**  
   - Balance the level of abstraction so that clusters are meaningful, interpretable, and useful for guiding product enhancements.
   - Avoid over-merging distinct topics or creating overly fragmented clusters.

4. **Output Format:**  
   - Respond with a JSON object in the exact format below, with no additional text:
  { "clusters": [ { "description": "short description of the cluster", "feature_requests": [list of feature request numbers that belong to this cluster] } // ... additional clusters ] }

### Input Format
- A numbered list of feature requests is provided below.
- Each feature request has a **SUBJECT** and a **DESCRIPTION**.

### Your Task
Based on the input, analyze the content and cluster the feature requests into groups with clear and actionable descriptions. Then, output the JSON structure with each cluster’s description and the list of feature request numbers that belong to it.

### Feature Requests
"""   


In [0]:
final_df.to_excel("Customer_Request_Cluster_200_points.xlsx", index=False)

In [0]:
backup_df = final_df.copy()

In [0]:
final_df.reset_index(drop=True, inplace=True)

In [0]:
final_df["BU"].value_counts()

Freshdesk       125
Freshservice     54
Freshchat        21
Name: BU, dtype: int64

In [0]:
final_df[final_df["BU"]=="Freshdesk"].shape[0]

125

In [0]:
#################
# For Freshchat
#################
n = final_df[final_df["BU"]=="Freshchat"].shape[0]
_freshchat_df = final_df[final_df["BU"]=="Freshchat"].copy()
_freshchat_df.reset_index(drop=True, inplace=True)

cluster_label_freshchat = [""]*n
_range = n//25
if n%25!=0:
  _range += 1

for iteration in range(_range):
  feature_requests = []
  # _texts = temp_df.head(100)["text_2_cluster"].tolist()
  _texts = _freshchat_df.iloc[(iteration*25):(iteration*25 + 25)]["formatted_text"].tolist()
  for i in range(len(_texts)):
    feature_requests.append("REQUEST "+str(i+1)+"\n---------- \n"+_texts[i]+"\n\n")
  messages = [
    {
      "role": "system",
      "content": prompt+"\n"+"\n".join(feature_requests)
    }
  ]
  raw_res = get_res_cloudverse_endpoint(messages=messages, temperature=0.3)
  parsed_res = extract_json(raw_res[0], inner_json=False)
  parsed_res = eval(parsed_res["clusters"])
  print("Parsed Response")
  print(parsed_res)
  print("**********")
  for item in parsed_res:
    _desc = item["description"].replace("Feature requests related to", "").strip().capitalize()
    for idx in item["feature_requests"]:
      # print((i*50)+(idx-1))
      cluster_label_freshchat[(iteration*25)+(idx-1)] = _desc
_freshchat_df["Cluster"] = cluster_label_freshchat

/databricks/python/lib/python3.11/site-packages/urllib3/connectionpool.py:1056: InsecureRequestWarning: Unverified HTTPS request is being made to host 'cloudverse.freshworkscorp.com'. Adding certificate verification is strongly advised. See: https://urllib3.readthedocs.io/en/1.26.x/advanced-usage.html#ssl-warnings
  warnings.warn(


Parsed Response
[{'description': 'Issues related to chat and conversation management', 'feature_requests': [1, 3, 5, 8, 9, 11, 19, 21]}, {'description': 'Feature requests and issues related to Freshchat settings and configurations', 'feature_requests': [2, 6, 12, 14, 15, 16, 17]}, {'description': 'Requests related to chatbot and AI agent functionalities', 'feature_requests': [4, 10, 18]}, {'description': 'Requests related to automation and scheduling', 'feature_requests': [7]}, {'description': 'Requests related to data management and reporting', 'feature_requests': [13, 20]}]
**********


In [0]:
_freshchat_df.display()

Ticket ID Subject Description Status Created Time Priority Source BU Category Sub-Category Item Full name Email CXG Segment Region Total MRR Freshservice MRR Freshdesk MRR Freshsales MRR Company Name Company Domains text_2_cluster formatted_text Cluster 17290987 Sunweb || Archived field showing up in Freshchat Sunweb || Archived field showing up in Freshchat Waiting on Feature Request 2025-01-06T19:29:08Z Medium Phone Freshchat Integrations Freshdesk Integrations Marine Christophe marine.christophe@sunwebgroup.com Top Tier EU 36386 36386.0 null null sunwebgroup.com sunwebgroup.com SUBJECT: Sunweb || Archived field showing up in Freshchat
DESCRIPTION: Sunweb || Archived field showing up in Freshchat **Subject:** Sunweb || Archived Field Showing Up in Freshchat

**Description:** Sunweb || Archived Field Showing Up in Freshchat Issues related to chat and conversation management 17306728 Freshchat CSAT How to setup and use CSAT in Freshchat  Waiting on Feature Request 2025-01-09T01:24:42Z Medium Phone Freshchat CSAT Customization CSAT Ahmer Altaf Budoo ahmer@freshprints.com Top Tier US 11925 null 7550.0 0.0 Fresh Prints freshprints.com SUBJECT: Freshchat CSAT
DESCRIPTION: How to setup and use CSAT in Freshchat  **Subject:** Freshchat CSAT 
**Description:** How to set up and use CSAT in Freshchat

---

**Setting Up CSAT in Freshchat:**

1. **Access Freshchat Settings:**
 - Log in to your Freshchat account.
 - Navigate to the settings section.

2. **Enable CSAT:**
 - Find the CSAT option in the settings.
 - Toggle the switch to enable CSAT surveys.

3. **Customize CSAT Survey:**
 - Choose the type of questions you want to include.
 - Customize the survey to match your brand's tone and style.

4. **Set Survey Triggers:**
 - Decide when the survey should be sent (e.g., after a conversation ends).
 - Configure the triggers accordingly.

5. **Save Changes:**
 - Review your settings and save the changes.

---

**Using CSAT in Freshchat:**

1. **Monitor Responses:**
 - Access the CSAT dashboard to view customer feedback.
 - Analyze the responses to gauge customer satisfaction.

2. **Respond to Feedback:**
 - Address any negative feedback promptly.
 - Use positive feedback to reinforce successful strategies.

3. **Adjust Strategies:**
 - Use insights from CSAT to improve customer service.
 - Implement changes based on feedback to enhance customer experience.

4. **Regular Review:**
 - Regularly review CSAT scores and feedback.
 - Continuously refine your approach to maintain high satisfaction levels.

By following these steps, you can effectively set up and utilize CSAT in Freshchat to improve customer satisfaction and service quality. Feature requests and issues related to freshchat settings and configurations 17310703 Edit contact information in Chat Widget Now I have another problem. When i open a chat or a ticket i cannot modify my contact detail. As you can see here i miss a lot of information i want to have from my contacts as the Owner and i have all blank fields for that. Since my crm colleagues are rewarded also for the number of contacts it's impossible for me to handle it Betty Festa 353.33.28.416   Whatsappami qui!   betty@wecanrace.it Lasciaci una Recensione  ⭐️⭐️⭐️⭐️⭐️ Waiting on Feature Request 2025-01-09T16:20:37Z Medium Phone Freshchat Inbox User/Custom user properties Inbox Betty Festa betty@wecanrace.it Digital Tier EU 390 null null null wecanrace.it wecanrace.it SUBJECT: Edit contact information in Chat Widget
DESCRIPTION: Now I have another problem. When i open a chat or a ticket i cannot modify my contact detail. As you can see here i miss a lot of information i want to have from my contacts as the Owner and i have all blank fields for that. Since my crm colleagues are rewarded also for the number of contacts it's impossible for me to handle it Betty Festa 353.33.28.416   Whatsappami qui!   betty@wecanrace.it Lasciaci una Recensione  ⭐️⭐️⭐️⭐️⭐️ **Subject:** Edit Contact Information in Chat Widget

**Description:** 

I

In [0]:
#################
# For Freshservice
#################
n = final_df[final_df["BU"]=="Freshservice"].shape[0]
_freshservice_df = final_df[final_df["BU"]=="Freshservice"].copy()
_freshservice_df.reset_index(drop=True, inplace=True)

cluster_label_freshservice = [""]*n
_range = n//25
if n%25!=0:
  _range += 1

for iteration in range(_range):
  feature_requests = []
  # _texts = temp_df.head(100)["text_2_cluster"].tolist()
  _texts = _freshservice_df.iloc[(iteration*25):(iteration*25 + 25)]["formatted_text"].tolist()
  for i in range(len(_texts)):
    feature_requests.append("REQUEST "+str(i+1)+"\n---------- \n"+_texts[i]+"\n\n")
  messages = [
    {
      "role": "system",
      "content": prompt+"\n"+"\n".join(feature_requests)
    }
  ]
  raw_res = get_res_cloudverse_endpoint(messages=messages, temperature=0.3)
  parsed_res = extract_json(raw_res[0], inner_json=False)
  parsed_res = eval(parsed_res["clusters"])
  print("Parsed Response")
  print(parsed_res)
  print("**********")
  for item in parsed_res:
    _desc = item["description"].replace("Feature requests related to", "").strip().capitalize()
    for idx in item["feature_requests"]:
      # print((i*50)+(idx-1))
      cluster_label_freshservice[(iteration*25)+(idx-1)] = _desc
_freshservice_df["Cluster"] = cluster_label_freshservice

/databricks/python/lib/python3.11/site-packages/urllib3/connectionpool.py:1056: InsecureRequestWarning: Unverified HTTPS request is being made to host 'cloudverse.freshworkscorp.com'. Adding certificate verification is strongly advised. See: https://urllib3.readthedocs.io/en/1.26.x/advanced-usage.html#ssl-warnings
  warnings.warn(


Parsed Response
[{'description': 'Email and Notification Management', 'feature_requests': [1, 20]}, {'description': 'Portal and Customization Features', 'feature_requests': [2, 5, 23]}, {'description': 'Agent and Ticket Management', 'feature_requests': [3, 6, 19]}, {'description': 'Holiday and Business Hours Management', 'feature_requests': [4]}, {'description': 'Knowledge Base and Solution Article Enhancements', 'feature_requests': [12, 18]}, {'description': 'Service Catalog and Field Customization', 'feature_requests': [8, 22, 25]}, {'description': 'Project and Task Management', 'feature_requests': [15]}, {'description': 'Onboarding and HR Management', 'feature_requests': [16]}, {'description': 'Hyperlink and Link Checker Features', 'feature_requests': [17]}, {'description': 'IP and Email Configuration Issues', 'feature_requests': [13]}, {'description': 'SLA and Priority Management', 'feature_requests': [14, 24]}, {'description': 'Tag and Metadata Management', 'feature_requests': [21

/databricks/python/lib/python3.11/site-packages/urllib3/connectionpool.py:1056: InsecureRequestWarning: Unverified HTTPS request is being made to host 'cloudverse.freshworkscorp.com'. Adding certificate verification is strongly advised. See: https://urllib3.readthedocs.io/en/1.26.x/advanced-usage.html#ssl-warnings
  warnings.warn(


Parsed Response
[{'description': 'Issues and feature requests related to asset management and tracking', 'feature_requests': [1, 12, 13, 21]}, {'description': 'Problems and enhancements related to notifications and email settings', 'feature_requests': [2, 16]}, {'description': 'Service catalog and workflow automation issues', 'feature_requests': [3, 5, 6, 9, 22]}, {'description': 'Employee onboarding and offboarding process improvements', 'feature_requests': [4, 11, 15]}, {'description': 'Requests for creating and managing child tickets', 'feature_requests': [8, 15]}, {'description': 'User permissions and role management issues', 'feature_requests': [7, 24]}, {'description': 'Requests for API enhancements and integrations', 'feature_requests': [19]}, {'description': 'Requests for UI/UX improvements in task and approval views', 'feature_requests': [10, 25]}, {'description': 'General feature requests and enhancements', 'feature_requests': [17, 18, 20, 23]}]
**********


/databricks/python/lib/python3.11/site-packages/urllib3/connectionpool.py:1056: InsecureRequestWarning: Unverified HTTPS request is being made to host 'cloudverse.freshworkscorp.com'. Adding certificate verification is strongly advised. See: https://urllib3.readthedocs.io/en/1.26.x/advanced-usage.html#ssl-warnings
  warnings.warn(


Parsed Response
[{'description': 'Knowledge Base Article Management and Security', 'feature_requests': [1, 2]}, {'description': 'Onboarding Process and Workflow Management', 'feature_requests': [3]}, {'description': 'AI Data Usage and Licensing', 'feature_requests': [4]}]
**********


In [0]:
_freshservice_df.display()

Ticket ID Subject Description Status Created Time Priority Source BU Category Sub-Category Item Full name Email CXG Segment Region Total MRR Freshservice MRR Freshdesk MRR Freshsales MRR Company Name Company Domains text_2_cluster formatted_text Cluster 17291643 Conversation with Alfred Riley - Disable email notifications for a specific requester group Alfred Riley How can i suppress notifications mails to external Clients 09:01 PM, 06th Jan Mercy Hi Alfred! I'm Mercy Bernadette, How may I assist you today? 09:02 PM, 06th Jan Alfred Riley how can i disable notifications for certain mails, mainly from external mail systems 09:02 PM, 06th Jan Mercy You can disable notifications for emails within a workspace based on the available in the list 09:04 PM, 06th Jan Mercy Admin > Workspace > Email notifications 09:04 PM, 06th Jan Mercy 09:05 PM, 06th Jan Alfred Riley I know this - i want to disable the notification only for certain requester groups, e.g. automated Amazon Mailboxes 09:06 PM, 06th Jan Mercy Unfortunatley this is not possible at the moment, but let me take this up with the backend team to hear if they have an alternative solution 09:08 PM, 06th Jan Alfred Riley This would be awesome - thx 09:09 PM, 06th Jan Mercy Most probably, they would consider this as a Feature Enhancement Request, But I will convert this chat into a ticket and keep you updated 09:09 PM, 06th Jan Mercy Is there anything else that I can assist you with? 09:10 PM, 06th Jan Alfred Riley Thank you, so far that's atm 09:10 PM, 06th Jan Alfred Riley *all 09:10 PM, 06th Jan Take to Team Inbox Waiting on Feature Request 2025-01-06T21:11:54Z Low Chat Freshservice Email Notification How To's Alfred Riley alfred.riley@craftview.de Mid Tier EU 896 896.0 null null craftview.de craftview.de SUBJECT: Conversation with Alfred Riley - Disable email notifications for a specific requester group
DESCRIPTION: Alfred Riley How can i suppress notifications mails to external Clients 09:01 PM, 06th Jan Mercy Hi Alfred! I'm Mercy Bernadette, How may I assist you today? 09:02 PM, 06th Jan Alfred Riley how can i disable notifications for certain mails, mainly from external mail systems 09:02 PM, 06th Jan Mercy You can disable notifications for emails within a workspace based on the available in the list 09:04 PM, 06th Jan Mercy Admin > Workspace > Email notifications 09:04 PM, 06th Jan Mercy 09:05 PM, 06th Jan Alfred Riley I know this - i want to disable the notification only for certain requester groups, e.g. automated Amazon Mailboxes 09:06 PM, 06th Jan Mercy Unfortunatley this is not possible at the moment, but let me take this up with the backend team to hear if they have an alternative solution 09:08 PM, 06th Jan Alfred Riley This would be awesome - thx 09:09 PM, 06th Jan Mercy Most probably, they would consider this as a Feature Enhancement Request, But I will convert this chat into a ticket and keep you updated 09:09 PM, 06th Jan Mercy Is there anything else that I can assist you with? 09:10 PM, 06th Jan Alfred Riley Thank you, so far that's atm 09:10 PM, 06th Jan Alfred Riley *all 09:10 PM, 06th Jan Take to Team Inbox **Subject:** Conversation with Alfred Riley - Disable email notifications for a specific requester group

**Description:**

**Alfred Riley:** How can I suppress notifications mails to external Clients? 
*09:01 PM, 06th Jan*

**Mercy:** Hi Alfred! I'm Mercy Bernadette, How may I assist you today? 
*09:02 PM, 06th Jan*

**Alfred Riley:** How can I disable notifications for certain mails, mainly from external mail systems? 
*09:02 PM, 06th Jan*

**Mercy:** You can disable notifications for emails within a workspace based on the available options in the list. 
*09:04 PM, 06th Jan*

**Mercy:** Admin > Workspace > Email notifications 
*09:04 PM, 06th Jan*

**Alfred Riley:** I know this - I want to disable the notification only for certain requester groups, e.g., automated Amazon Mailboxes. 
*09:06 PM, 06th Jan*

**Mercy:** Unfortunately, this is not possible at 

In [0]:
#################
# For Freshdesk
#################
n = final_df[final_df["BU"]=="Freshdesk"].shape[0]
_freshdesk_df = final_df[final_df["BU"]=="Freshdesk"].copy()
_freshdesk_df.reset_index(drop=True, inplace=True)

cluster_label_freshdesk = [""]*n
_range = n//25
if n%25!=0:
  _range += 1

for iteration in range(_range):
  feature_requests = []
  # _texts = temp_df.head(100)["text_2_cluster"].tolist()
  _texts = _freshdesk_df.iloc[(iteration*25):(iteration*25 + 25)]["formatted_text"].tolist()
  for i in range(len(_texts)):
    feature_requests.append("REQUEST "+str(i+1)+"\n---------- \n"+_texts[i]+"\n\n")
  messages = [
    {
      "role": "system",
      "content": prompt_v2+"\n"+"\n".join(feature_requests)
    }
  ]
  raw_res = get_res_cloudverse_endpoint(messages=messages, temperature=0.3)
  parsed_res = extract_json(raw_res[0], inner_json=False)
  parsed_res = eval(parsed_res["clusters"])
  print("Parsed Response")
  print(parsed_res)
  print("**********")
  for item in parsed_res:
    _desc = item["description"].replace("Feature requests related to", "").strip().capitalize()
    for idx in item["feature_requests"]:
      # print((i*50)+(idx-1))
      cluster_label_freshdesk[(iteration*25)+(idx-1)] = _desc
_freshdesk_df["Cluster"] = cluster_label_freshdesk

/databricks/python/lib/python3.11/site-packages/urllib3/connectionpool.py:1056: InsecureRequestWarning: Unverified HTTPS request is being made to host 'cloudverse.freshworkscorp.com'. Adding certificate verification is strongly advised. See: https://urllib3.readthedocs.io/en/1.26.x/advanced-usage.html#ssl-warnings
  warnings.warn(


Parsed Response
[{'description': 'Ticket sorting and filtering enhancements', 'feature_requests': [1, 3, 22]}, {'description': 'Analytics and reporting improvements', 'feature_requests': [2, 4, 18]}, {'description': 'Integration issues and enhancements', 'feature_requests': [5, 10, 19]}, {'description': 'User interface and customization requests', 'feature_requests': [12, 13, 24]}, {'description': 'SLA and escalation customization', 'feature_requests': [8, 9]}, {'description': 'Portal and branding customization', 'feature_requests': [11, 17]}, {'description': 'Translation and localization issues', 'feature_requests': [20]}, {'description': 'Automation and workflow issues', 'feature_requests': [21, 23, 25]}, {'description': 'Ticket merging and contact management', 'feature_requests': [7, 16]}, {'description': 'Business hours and scheduling features', 'feature_requests': [14]}, {'description': 'Request for additional portals and multi-brand support', 'feature_requests': [17]}]
**********

/databricks/python/lib/python3.11/site-packages/urllib3/connectionpool.py:1056: InsecureRequestWarning: Unverified HTTPS request is being made to host 'cloudverse.freshworkscorp.com'. Adding certificate verification is strongly advised. See: https://urllib3.readthedocs.io/en/1.26.x/advanced-usage.html#ssl-warnings
  warnings.warn(


Parsed Response
[{'description': 'User Interface and Usability Enhancements', 'feature_requests': [1, 5, 13, 25]}, {'description': 'Export and Data Management Issues', 'feature_requests': [2, 3, 19]}, {'description': 'Language and Localization Support', 'feature_requests': [4, 11]}, {'description': 'Email and Notification Management', 'feature_requests': [6, 22]}, {'description': 'Custom Views and Filtering in Ticket Management', 'feature_requests': [7, 23]}, {'description': 'Analytics and Reporting Customization', 'feature_requests': [8, 12, 18]}, {'description': 'SLA and Field Customization', 'feature_requests': [10, 17]}, {'description': 'Agent and Collaborator Role Management', 'feature_requests': [21, 24]}, {'description': 'Duplicate Tickets and Mandatory Fields', 'feature_requests': [16]}, {'description': 'Feature Request Tracking', 'feature_requests': [14, 15]}, {'description': 'Ticketing System Bugs and Issues', 'feature_requests': [9, 20]}]
**********


/databricks/python/lib/python3.11/site-packages/urllib3/connectionpool.py:1056: InsecureRequestWarning: Unverified HTTPS request is being made to host 'cloudverse.freshworkscorp.com'. Adding certificate verification is strongly advised. See: https://urllib3.readthedocs.io/en/1.26.x/advanced-usage.html#ssl-warnings
  warnings.warn(


Parsed Response
[{'description': 'Issues with mandatory fields and ticket properties', 'feature_requests': [1]}, {'description': 'Customization and filtering options in customer portal views', 'feature_requests': [2]}, {'description': 'Automation and integration requests with third-party applications', 'feature_requests': [3, 17]}, {'description': 'Managing and customizing ticket fields and default settings', 'feature_requests': [4, 13]}, {'description': 'Ticket reopening and status management', 'feature_requests': [5]}, {'description': 'CSAT survey limitations and feedback management', 'feature_requests': [6]}, {'description': 'Social media and external communication integrations', 'feature_requests': [7]}, {'description': 'Mobile app feature requests and feedback', 'feature_requests': [8]}, {'description': 'Permissions and access control for external users', 'feature_requests': [9]}, {'description': 'Knowledge base and article management improvements', 'feature_requests': [10, 11]}, 

/databricks/python/lib/python3.11/site-packages/urllib3/connectionpool.py:1056: InsecureRequestWarning: Unverified HTTPS request is being made to host 'cloudverse.freshworkscorp.com'. Adding certificate verification is strongly advised. See: https://urllib3.readthedocs.io/en/1.26.x/advanced-usage.html#ssl-warnings
  warnings.warn(


Parsed Response
[{'description': 'Export and Data Management Features', 'feature_requests': [1, 2, 6, 12, 24]}, {'description': 'Analytics and Reporting Enhancements', 'feature_requests': [4, 5, 7, 17]}, {'description': 'Ticket Management and Workflow Improvements', 'feature_requests': [3, 13, 15, 22]}, {'description': 'Mobile App Functionality Requests', 'feature_requests': [16, 19, 23]}, {'description': 'Email and Communication Issues', 'feature_requests': [8, 10]}, {'description': 'User Interface and Usability Enhancements', 'feature_requests': [9, 11, 21]}, {'description': 'Automation and AI Capabilities', 'feature_requests': [18, 20]}, {'description': 'Notification and Template Customization', 'feature_requests': [14, 25]}]
**********


/databricks/python/lib/python3.11/site-packages/urllib3/connectionpool.py:1056: InsecureRequestWarning: Unverified HTTPS request is being made to host 'cloudverse.freshworkscorp.com'. Adding certificate verification is strongly advised. See: https://urllib3.readthedocs.io/en/1.26.x/advanced-usage.html#ssl-warnings
  warnings.warn(


Parsed Response
[{'description': 'Dark Mode Feature Requests', 'feature_requests': [1, 12]}, {'description': 'Permission and Access Control Enhancements', 'feature_requests': [2, 6]}, {'description': 'Automation and Workflow Improvements', 'feature_requests': [3, 4, 11, 14, 16]}, {'description': 'CSAT and Survey Management', 'feature_requests': [5, 25]}, {'description': 'WhatsApp Integration and Features', 'feature_requests': [8, 10]}, {'description': 'Knowledge Base and Content Management', 'feature_requests': [15, 17]}, {'description': 'Reporting and Analytics Enhancements', 'feature_requests': [21, 23, 25]}, {'description': 'Contact Management and Export Features', 'feature_requests': [22]}, {'description': 'File Attachment and Preview Features', 'feature_requests': [9, 20]}, {'description': 'Custom Fields and Filtering', 'feature_requests': [18]}, {'description': 'Security and Data Handling', 'feature_requests': [24]}, {'description': 'Canned Responses and Easy Customization', 'fea

In [0]:
pd.concat([_freshservice_df, _freshchat_df, _freshdesk_df]).to_excel("Customer_Request_Cluster_200_points_v2.xlsx", index=False)

In [0]:
print(messages[0]["content"])

You are am expert product manager working in CRM and ITSM. Given below are some feature requests from customers. Your task is to cluster these together and provide a short description of each cluster. Also output the feature request number that you think belongs to each cluster.

### Input
- Inout contains a numbered list of feature requests.
- Each feature request has a SUBJECT and a DESCRIPTION
- The DESCRIPTION can be one of the below:
  - A paragraph/email describing the feature that the customer wants
  - A chat between the customer and 'a bot or a human agent' where the customer is describing what their issue is and what feature they would want.

### Important Points to Note
- ENSURE each feature request is assigned a cluster.
- Cluster the given data points into meaningful groups based on their semantic similarities. Ensure that the clusters are neither too broad nor overly granular, capturing key distinctions while maintaining coherence within each group. 
- The segmentation sh

In [0]:
len([x for x in cluster_label if x])

172

In [0]:
final_df["Cluster"] = cluster_label

In [0]:
final_df.to_excel("Customer_Request_Cluster_200_points_v2.xlsx", index=False)

In [0]:
final_df.display()

Ticket ID Subject Description Status Created Time Priority Source BU Category Sub-Category Item Full name Email CXG Segment Region Total MRR Freshservice MRR Freshdesk MRR Freshsales MRR Company Name Company Domains text_2_cluster formatted_text Cluster 17290987 Sunweb || Archived field showing up in Freshchat Sunweb || Archived field showing up in Freshchat Waiting on Feature Request 2025-01-06T19:29:08Z Medium Phone Freshchat Integrations Freshdesk Integrations Marine Christophe marine.christophe@sunwebgroup.com Top Tier EU 36386 36386.0 null null sunwebgroup.com sunwebgroup.com SUBJECT: Sunweb || Archived field showing up in Freshchat
DESCRIPTION: Sunweb || Archived field showing up in Freshchat **Subject:** Sunweb || Archived Field Showing Up in Freshchat

**Description:** Sunweb || Archived Field Showing Up in Freshchat 17291643 Conversation with Alfred Riley - Disable email notifications for a specific requester group Alfred Riley How can i suppress notifications mails to external Clients 09:01 PM, 06th Jan Mercy Hi Alfred! I'm Mercy Bernadette, How may I assist you today? 09:02 PM, 06th Jan Alfred Riley how can i disable notifications for certain mails, mainly from external mail systems 09:02 PM, 06th Jan Mercy You can disable notifications for emails within a workspace based on the available in the list 09:04 PM, 06th Jan Mercy Admin > Workspace > Email notifications 09:04 PM, 06th Jan Mercy 09:05 PM, 06th Jan Alfred Riley I know this - i want to disable the notification only for certain requester groups, e.g. automated Amazon Mailboxes 09:06 PM, 06th Jan Mercy Unfortunatley this is not possible at the moment, but let me take this up with the backend team to hear if they have an alternative solution 09:08 PM, 06th Jan Alfred Riley This would be awesome - thx 09:09 PM, 06th Jan Mercy Most probably, they would consider this as a Feature Enhancement Request, But I will convert this chat into a ticket and keep you updated 09:09 PM, 06th Jan Mercy Is there anything else that I can assist you with? 09:10 PM, 06th Jan Alfred Riley Thank you, so far that's atm 09:10 PM, 06th Jan Alfred Riley *all 09:10 PM, 06th Jan Take to Team Inbox Waiting on Feature Request 2025-01-06T21:11:54Z Low Chat Freshservice Email Notification How To's Alfred Riley alfred.riley@craftview.de Mid Tier EU 896 896.0 null null craftview.de craftview.de SUBJECT: Conversation with Alfred Riley - Disable email notifications for a specific requester group
DESCRIPTION: Alfred Riley How can i suppress notifications mails to external Clients 09:01 PM, 06th Jan Mercy Hi Alfred! I'm Mercy Bernadette, How may I assist you today? 09:02 PM, 06th Jan Alfred Riley how can i disable notifications for certain mails, mainly from external mail systems 09:02 PM, 06th Jan Mercy You can disable notifications for emails within a workspace based on the available in the list 09:04 PM, 06th Jan Mercy Admin > Workspace > Email notifications 09:04 PM, 06th Jan Mercy 09:05 PM, 06th Jan Alfred Riley I know this - i want to disable the notification only for certain requester groups, e.g. automated Amazon Mailboxes 09:06 PM, 06th Jan Mercy Unfortunatley this is not possible at the moment, but let me take this up with the backend team to hear if they have an alternative solution 09:08 PM, 06th Jan Alfred Riley This would be awesome - thx 09:09 PM, 06th Jan Mercy Most probably, they would consider this as a Feature Enhancement Request, But I will convert this chat into a ticket and keep you updated 09:09 PM, 06th Jan Mercy Is there anything else that I can assist you with? 09:10 PM, 06th Jan Alfred Riley Thank you, so far that's atm 09:10 PM, 06th Jan Alfred Riley *all 09:10 PM, 06th Jan Take to Team Inbox **Subject:** Conversation with Alfred Riley - Disable email notifications for a specific requester group

**Description:**

**Alfred Riley:** How can I suppress notifications mails to external Clients? 
*09:01 PM, 06th Jan*

**Mercy:** Hi Alfred! I'm Mercy Bernadette, How may I assist

## Get Feature Summary

In [0]:
input_data_df = pd.read_excel("Customer_Request_Cluster_200_points_v3.xlsx")

In [0]:
data_df = input_data_df[["Ticket ID", "BU", "formatted_text", "Final_Cluster"]].copy()

In [0]:
prompt_feature_summary = """Given below are conversations between customer and an agent. The cutomer is describing some problem and asking for certain feature(s) to be added to the product which could be entirely new feature or an enhancement.
Your task is to analyse these conversations and come up with a feature and a short description for that feature based on customer requirements. 

### Instructions
- Club together similar requests to come up with a consolidated feature.
- The generated feature name and description would be consumed by executives so should be short and concise but explains well what that feature is.
- For each feature that you suggest, also cite the conversation id on which the feature is based.

### Response Format
Always output the below JSON without any additional text:
{{
  "features": [{{
    "feature_name": "short title for the feature",
    "feature_description": "short description for the feature",
    "conversation_id": [<list of conversations on which this feature is based>]
  }},
  ...
}}

### Conversations
{conversations}
"""

In [0]:
bu = data_df["BU"].unique().tolist()

In [0]:
data_df["BU"].value_counts()

Freshdesk       125
Freshservice     54
Freshchat        21
Name: BU, dtype: int64

In [0]:
bu

['Freshservice', 'Freshchat', 'Freshdesk']

In [0]:
results = {
  'Freshservice': [],
  'Freshchat': [],
  'Freshdesk': []
}

In [0]:
for _bu in bu[-1:]:
  _df = data_df[data_df["BU"] == _bu].copy()
  clusters = _df["Final_Cluster"].unique().tolist()
  for _cluster in clusters:
    _df_cluster = _df[_df["Final_Cluster"] == _cluster].copy()
    _conv = []
    for idx, row in _df_cluster.iterrows():
      _conv.append("Conversation "+str(row["Ticket ID"])+"\n----------------\n"+row["formatted_text"]+"\n")
    messages = [
      {
        "role": "system",
        "content": prompt_feature_summary.format(conversations="".join(_conv))
      }
    ]
    raw_res = get_res_cloudverse_endpoint(messages=messages, temperature=0.3)[0]
    print("BU: {}, Cluster {}".format(_bu, _cluster))
    print(raw_res)
    results[_bu].append({_cluster: extract_json(raw_res, inner_json=False)})

/databricks/python/lib/python3.11/site-packages/urllib3/connectionpool.py:1056: InsecureRequestWarning: Unverified HTTPS request is being made to host 'cloudverse.freshworkscorp.com'. Adding certificate verification is strongly advised. See: https://urllib3.readthedocs.io/en/1.26.x/advanced-usage.html#ssl-warnings
  warnings.warn(


BU: Freshdesk, Cluster Ticket sorting and filtering enhancements
```json
{
  "features": [
    {
      "feature_name": "Logical Ticket Sorting",
      "feature_description": "Enable sorting of tickets by most recent updates or creation date for better visibility and management.",
      "conversation_id": [17292446]
    },
    {
      "feature_name": "Default Custom View",
      "feature_description": "Allow users to set a default custom filter view for tickets upon opening Freshdesk, instead of defaulting to all tickets.",
      "conversation_id": [17297638]
    },
    {
      "feature_name": "Ticket Merging and De-Merging",
      "feature_description": "Provide functionality to merge tickets based on search results and de-merge contacts that were wrongly merged.",
      "conversation_id": [17301966, 17311401]
    },
    {
      "feature_name": "Filter by System Flags",
      "feature_description": "Enable filtering of tickets based on system flags like 'Customer responded' to enhance 

/databricks/python/lib/python3.11/site-packages/urllib3/connectionpool.py:1056: InsecureRequestWarning: Unverified HTTPS request is being made to host 'cloudverse.freshworkscorp.com'. Adding certificate verification is strongly advised. See: https://urllib3.readthedocs.io/en/1.26.x/advanced-usage.html#ssl-warnings
  warnings.warn(


BU: Freshdesk, Cluster Analytics and reporting improvements
```json
{
  "features": [
    {
      "feature_name": "Solution Article View Count Accuracy",
      "feature_description": "Ensure accurate tracking and representation of solution article view counts to measure adoption and usage effectively.",
      "conversation_id": [17296374]
    },
    {
      "feature_name": "Donut Chart Percentage Toggle",
      "feature_description": "Provide an option to remove or toggle percentages in donut chart legends for customized report views.",
      "conversation_id": [17297860]
    },
    {
      "feature_name": "Custom Priority Sorting in Analytics",
      "feature_description": "Allow users to customize the sorting order of priorities in analytics reports.",
      "conversation_id": [17312728]
    },
    {
      "feature_name": "Expanded CSAT Survey Scale",
      "feature_description": "Increase the customer satisfaction survey scale beyond 7 levels to accommodate more granular feedback.",

/databricks/python/lib/python3.11/site-packages/urllib3/connectionpool.py:1056: InsecureRequestWarning: Unverified HTTPS request is being made to host 'cloudverse.freshworkscorp.com'. Adding certificate verification is strongly advised. See: https://urllib3.readthedocs.io/en/1.26.x/advanced-usage.html#ssl-warnings
  warnings.warn(


...

BU: Freshdesk, Cluster Csat survey limitations and feedback management
```json
{
  "features": [
    {
      "feature_name": "Limit CSAT Survey Submissions",
      "feature_description": "Restrict customers to submit only one CSAT survey per ticket to prevent multiple feedback entries for the same issue.",
      "conversation_id": [17363163]
    },
    {
      "feature_name": "CSAT Survey Expiration",
      "feature_description": "Introduce an option to automatically expire CSAT surveys after a specified number of days to ensure timely feedback.",
      "conversation_id": [17413928]
    }
  ]
}
```


/databricks/python/lib/python3.11/site-packages/urllib3/connectionpool.py:1056: InsecureRequestWarning: Unverified HTTPS request is being made to host 'cloudverse.freshworkscorp.com'. Adding certificate verification is strongly advised. See: https://urllib3.readthedocs.io/en/1.26.x/advanced-usage.html#ssl-warnings
  warnings.warn(


BU: Freshdesk, Cluster Knowledge base and article management improvements
```json
{
  "features": [
    {
      "feature_name": "Clickable Tags with Article Grouping",
      "feature_description": "Enable article tags to be clickable, allowing users to view all articles associated with a specific tag directly from the article detail page.",
      "conversation_id": [17365770]
    },
    {
      "feature_name": "Display Full Path in Search Results",
      "feature_description": "Enhance search functionality to display the full folder path in search results, improving clarity and organization when multiple folders have similar names.",
      "conversation_id": [17365770, 17365824]
    },
    {
      "feature_name": "Article Approval Workflow",
      "feature_description": "Implement an approval process for new articles in the knowledge base, requiring content admin approval before publication to ensure content quality and consistency.",
      "conversation_id": [17426550]
    },
    {
  

/databricks/python/lib/python3.11/site-packages/urllib3/connectionpool.py:1056: InsecureRequestWarning: Unverified HTTPS request is being made to host 'cloudverse.freshworkscorp.com'. Adding certificate verification is strongly advised. See: https://urllib3.readthedocs.io/en/1.26.x/advanced-usage.html#ssl-warnings
  warnings.warn(


BU: Freshdesk, Cluster Contact list customization and management
```json
{
  "features": [
    {
      "feature_name": "Editable Contact List Rows",
      "feature_description": "Allow users to edit rows in the contact list for better customization and data management.",
      "conversation_id": [17374558]
    },
    {
      "feature_name": "Expanded Kanban Ticket Display",
      "feature_description": "Enable users to display all open tickets on the Kanban board without pagination, with options to filter by ticket status and date.",
      "conversation_id": [17397416]
    },
    {
      "feature_name": "Custom Email Notification Settings",
      "feature_description": "Provide users with the ability to select which email notifications they wish to receive through the customer portal.",
      "conversation_id": [17408820]
    }
  ]
}
```


In [0]:
ticket_feat = {}
for _bu in bu:
  for item in results[_bu]:
    # print(item.keys())
  
    val = list(item.values())[0]
    # print(val["features"])

    for x in eval(val["features"]):
      _f = x["feature_name"]
      _d = x["feature_description"]
      tid = x["conversation_id"]
      for t in tid:
        ticket_feat[t] = (_f, _d)

In [0]:
_feature = [""]*len(ticket_id)
_desc = [""]*len(ticket_id)
for idx in range(len(ticket_id)):
  if ticket_id[idx] in ticket_feat:
    _feature[idx], _desc[idx] = ticket_feat[ticket_id[idx]]

In [0]:
pd.DataFrame({"Ticket ID": ticket_id, "Feature": _feature, "Description": _desc}).display()

Ticket ID,Feature,Description
17291643,Custom Email Notification Suppression,Allow users to disable email notifications for specific requester groups or external systems.
17298944,Clone Portals,Enable the ability to clone existing portals for easier setup and management.
17299629,Archive Agent Comments,Enable retention of comments and notes from deleted agents to preserve historical troubleshooting and solutions.
17300359,Yearly Holiday Import Clarification,"Provide clear guidelines on importing yearly holiday lists, including handling holidays spanning across years and SLA timer management."
17300570,Problem Management in Business Workspaces,Introduce problem management capabilities within business workspaces for IT and business agents.
17301965,Restrict Ticket Creation by Email,"Introduce an option to restrict ticket creation based on specific email addresses, in addition to domain-based restrictions."
17306833,Subject Filter in Ticket List,"Add the ability to filter tickets by subject in the web-based ticket list, allowing users to save these filters for future use."
17308106,Pre-filled Field Properties,Allow fields in service catalog forms to be pre-filled with specific values for easier data entry.
17308108,Average First Assign Time Metric,Provide documentation and support for calculating the average first assign time for tickets.
17308203,Mobile Asset Attachment,"Enable the ability to attach files, such as pictures, to assets when creating or updating them via the mobile app."


In [0]:
data_df.shape

(200, 4)

In [0]:
df_2_write = pd.merge(input_data_df, pd.DataFrame({"Ticket ID": ticket_id, "Feature": _feature, "Description": _desc}), on="Ticket ID")

In [0]:
df_2_write.shape

(200, 28)

In [0]:
df_2_write.to_excel("Customer_Request_Cluster_200_points_v4.xlsx", index=False)